In [2]:
### Limpieza del Archivo de Farmacovigilancia RAM

In [3]:
import pandas as pd
from datetime import timedelta
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer,MissingIndicator 

Farmaco_RAM=pd.read_excel("../datos originales/Farmacovigilancia RAM.xlsx")

In [4]:
### Renombramiento de columnas segun el estilo acordado

Farmaco_RAM.columns

Index(['FECHA_NOTIFICACION', 'DEPARTAMENTO', 'MUNICIPIO', 'EPS', 'ETNIA',
       'FECHA_NACIMIENTO', 'EDAD', 'TIPO_IDENTIFICACION', 'NRO_IDENTIFICACION',
       'SEXO', 'PESO', 'ESTATURA', 'DIAGNOSTICO_PPAL_CONDICIONES_CLINICAS',
       'MEDICAMENTO_SOSPECHOSO', 'INDICACION', 'FECHA_INICIO_TRATAMIENTO',
       'FECHA_FIN_TRATAMIENTO', 'COMERCIAL', 'FECHA_INICIO_REACCION',
       'RAM_SOSPECHADA', 'SISTEMA_COMPROMETIDO', 'ANALISIS', 'EVOLUCION',
       'SERIEDAD', 'EVENTO_PRESENTO_DESPUES_ADMINISTRAR_MEDICAMENTO',
       'EXISTEN_OTROS_FACTORES_PUEDAN_EXPLICAR_EVENTO',
       'EVENTO_DESAPARECIO_DISMINUIR_SUSP_MED',
       'PACIENTE_HABIA_PRESENTADO_MISMA_REACION_MEDIC',
       'CAUSALIDAD_SEGUN_ALGORITMO_DE_NARANJO', 'EVITABILIDAD', 'GRAVEDAD',
       'DS_PLAN_INTERVENCION'],
      dtype='object')

In [5]:
Farmaco_RAM.rename(columns={"NRO_IDENTIFICACION":"ID_PACIENTE"},inplace=True)


In [6]:
### Revision Columnas

## ID Paciente
print(Farmaco_RAM.ID_PACIENTE.drop_duplicates().count())
print(Farmaco_RAM.ID_PACIENTE.isna().sum())
print(Farmaco_RAM.ID_PACIENTE.isnull().sum())

217
0
0


In [7]:
### Fechas

Farmaco_RAM.FECHA_NOTIFICACION=pd.to_datetime(Farmaco_RAM.FECHA_NOTIFICACION)
Farmaco_RAM.FECHA_NACIMIENTO=pd.to_datetime(Farmaco_RAM.FECHA_NACIMIENTO)
Farmaco_RAM.FECHA_INICIO_TRATAMIENTO=pd.to_datetime(Farmaco_RAM.FECHA_INICIO_TRATAMIENTO)
Farmaco_RAM.FECHA_FIN_TRATAMIENTO=pd.to_datetime(Farmaco_RAM.FECHA_FIN_TRATAMIENTO)
Farmaco_RAM.FECHA_INICIO_REACCION=pd.to_datetime(Farmaco_RAM.FECHA_INICIO_REACCION)
print(Farmaco_RAM.FECHA_NOTIFICACION.min())
print(Farmaco_RAM.FECHA_NOTIFICACION.max())
print(Farmaco_RAM.FECHA_NOTIFICACION.isna().sum())
print(Farmaco_RAM.FECHA_NOTIFICACION.isnull().sum())

2013-03-05 00:00:00
2020-05-06 00:00:00
0
0


In [8]:
### ID Caso debido a que un caso se va a separar en varias columnas

Farmaco_RAM=Farmaco_RAM.reset_index()
Farmaco_RAM.rename(columns={"index":"ID_RAM"},inplace=True)

In [9]:
### Separecion de Diagnostico principal condiciones clinicas en una fila por cada condicion 

Diagnosticos=Farmaco_RAM.DIAGNOSTICO_PPAL_CONDICIONES_CLINICAS.str.split("/",n=36,expand=True)
Unida=pd.concat([Farmaco_RAM,Diagnosticos],axis=1)

In [10]:

Fijas=Unida.columns[-36:]
Mover=Unida.columns[:-36]
Unida=Unida.melt(id_vars=Mover,value_name="DiagnosticoPpalCondicionesClinicas",var_name="NumDiagnostico")
Unida=Unida[~pd.isna(Unida["DiagnosticoPpalCondicionesClinicas"])]


In [11]:
## Indicacion y Diagnostico Separar el código de la Indicación/Diagnostico

abc=Unida.DiagnosticoPpalCondicionesClinicas.str.rsplit("[",expand=True,n=1)
abc[0]=abc[0].str.strip()
abc[1]=abc[1].str.replace("]","").str.strip()
abc.columns=["DiagnosticoPpalCondicionesClinicas","DiagnosticoPpalCondicionesClinicasCodigo"]
Unida.drop(columns="DiagnosticoPpalCondicionesClinicas",inplace=True)
Unida=pd.concat([Unida,abc],axis=1)



abc=Unida.INDICACION.str.rsplit("[",expand=True,n=1)
abc[0]=abc[0].str.strip()
abc[1]=abc[1].str.replace("]","").str.strip()
abc.columns=["Indicacion","IndicacionCodigo"]
Unida=pd.concat([Unida,abc],axis=1)
Farmaco_RAM=Unida


In [12]:
Farmaco_RAM.to_excel("Farmaco_RAM_limpia_18_jun.xlsx")